In [15]:
from datasets import load_dataset

ds = load_dataset("d0rj/wikisum")

In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 35775
    })
    validation: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 2000
    })
})

In [11]:
print(ds['train'][0]['article'])

Do not shuck or wash your oysters. Oysters taste best when you shuck them immediately before eating them. In addition, keeping oysters in their shells makes them easier to store and reduces the chance that they'll go bad. If your oysters came pre-shucked in a plastic container, store them in the freezer until you're ready to use them. Leave the grit and dirt on the oysters. This will keep them moist and will help to insulate the meat. Pour ice into a small bowl or other open-top container. Grab a bowl, small cooler, or similar container that you can place inside your fridge. Make sure this container has an open top or removable lid. Then, pour a layer of ice into the bottom of the container. Do not keep your oysters in a sealed or closed-top container. Doing so will suffocate them. You may need to change your ice during the refrigeration process, so do not pour any into the container if you won't be able to check your oysters regularly. Place your oysters on top of the ice bed deep sid

In [17]:
from tqdm import tqdm
def create_example(all_data):
    all_result = []
    data_id = 0
    for data in tqdm(all_data):
        result = {}
        result["_id"] = data_id
        result["question"] = "summary"
        result["document"] = data['article']
        result["output"] = data["summary"]
        all_result.append(result)
    return all_result

In [18]:
input_data = create_example(ds['train'])

100%|██████████| 35775/35775 [00:01<00:00, 24885.69it/s]


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_path = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer.padding_side = 'left'

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.53s/it]


In [20]:
from tqdm import tqdm
all_len = []
all_result = []

for input_data_ in tqdm(input_data):
    messages = [
        {"role": "system", "content": input_data_["question"]}, 
        {"role": "user", "content": input_data_["document"]},
        {"role":"assistant", "content": input_data_["output"]},
    ]
    count = len(tokenizer.apply_chat_template(messages))
    if count <= 2044:
        all_result.append(input_data_)
    all_len.append(count)

100%|██████████| 35775/35775 [01:51<00:00, 322.24it/s]


In [21]:
print(len(input_data))
print(len(all_result))

35775
25557


In [22]:
import json
with open("../data/1008data/wikisum_train.json", "w", encoding="utf-8") as f:
    json.dump(all_result, f, ensure_ascii=False, indent=4)